In [15]:
from tkinter import *
from tkinter import messagebox
import requests
import random

url = "https://the-trivia-api.com/api/questions?categories=film_and_tv&limit=10&region=US&difficulty=easy"
get_request = requests.get(url).json()

root = Tk()
root.title("Trivia")
root.geometry("1000x800")

# creating canvas
canvas = Canvas(root, width=800, height=250, bg="khaki1")
canvas.place(relx=.5, rely=.5, anchor=CENTER) # anchors the canvas in the center

# adding text to the canvas
game_display = canvas.create_text(200, 75, text="", fill="black", font=('Helvetica 15 bold'))

# creating the label that will display if something is correct/incorrect
feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))


score_label = Label(root, text="Score: 0", pady=10, font=("arial", 15, "bold"))
score_label.pack()

user_answer = StringVar()
user_answer.set("")

score = 0
question_counter = 0
question_limit = 10

def generate_qlist(json):
    question_list = []
    for i in json:
        i = i["question"]
        question_list.append(i)
    return question_list

def gen_question(list_):
    choice = random.choice(list_)
    return choice

def generate_answers(json, question): 
    option_list = []
    for i in get_request:
        if i["question"] == question:
            cor = i["correctAnswer"]
            option_list.append(cor)
            for incor in i["incorrectAnswers"]:
                option_list.append(incor)
    random.shuffle(option_list)
    return option_list

def ask_question(question, answers):
    canvas.itemconfig(game_display, text=question, width=300)
    
def gen_radiobuttons(answers):
    y_pos = 100
    for i in answers:
        rad_btn = Radiobutton(canvas, text=i, variable=user_answer, command=eval_score, value=i, font=('Helvetica 10 bold'))
        rad_btn.place(x=50, y=y_pos)
        y_pos+=20
    
def eval_score():
    answer = user_answer.get()
    for i in get_request:
        if i["question"] == current_question:
            if i["correctAnswer"] == answer:
                feedback["text"] = "Correct!"
                feedback.place(x=400, y=500)
                global score
                score += 10
                score_label["text"] = f"Score: {score}"
            else:
                feedback["text"] = "incorrect :("
                feedback.place(x=400, y=500)
    next_question()
    
def next_question():
    global question_counter
    question_counter += 1
    if question_counter == question_limit:
        messagebox.showinfo("Game Over", f"You got {score} out of {question_limit*10} points!")
        root.destroy()
    else:
        main()
        
def main(): 
    global current_question
    q_list = generate_qlist(get_request)
    current_question = gen_question(q_list)
    ans_list = generate_answers(get_request, current_question)
    ask_question(current_question, ans_list)
    gen_radiobuttons(ans_list)
    
    
main()

root.mainloop()
